# Проект 5 - Компьютер говорит нет

Заработок денег- это основная цель любого бизнес-проекта и банки не являются исключением. Если банк будет отказывать в кредите хорошему заемщику — ошибка первого рода. Но если банк даст кредит плохому заемщику, который потом его не отдаст, то банк потеряет деньги - это ошибка второго рода.
	
Для прогнозирования вероятности невозврата кредита банки используют кредитный скоринг - систему оценки кредитоспособности. Само слово скоринг произошло от английского «score» — подсчет очков. В ситуации с банками кредитные учреждения ведут «подсчет очков», оценивая платежеспособность клиентов. По этой системе идет классификация вероятностей плохих и хороших клиентов. Используя эту систему работник банка может понять, выдавать ли клиенту кредит или нет. Банки сами определяют для себя значения, при которых они принимают решение об отказе в кредите.

Цель данного проекта - выбрать наиболее эффективную модель для оценки качества клиентов банка.

In [111]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from catboost import CatBoost, CatBoostClassifier, Pool
from catboost.utils import get_roc_curve
import lightgbm
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

# settings to display all columns
pd.set_option("display.max_columns", None)

RANDOM_SEED = 42

In [112]:
def get_boxplot(df, col):
    fig, axes = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='default', y=col, data=df[df['sample']==1], ax=axes)
    axes.set_title('Boxplot for ' + col)
    plt.show()
    
    
def age_to_cat(age):
    if age <= 28:
        cat_age = 0
        return cat_age             
    if 28 < age <= 35:
        cat_age = 1
        return cat_age
    if 35 < age <= 50:
        cat_age = 2
        return cat_age
    if age > 50:
        cat_age = 3
        return cat_age
    
    
def show_metrics(y_test, y_pred, probs):
    print('accuracy_score:\t\t {:.4}'.format(accuracy_score(y_test, y_pred)))
    print('precision_score:\t {:.4}'.format(precision_score(y_test, y_pred, zero_division=0)))
    print('recall_score:\t\t {:.4}'.format(recall_score(y_test, y_pred, zero_division=0)))
    print('f1_score:\t\t {:.4}'.format(f1_score(y_test, y_pred, zero_division=0)))
    print('roc_auc_score:\t\t {:.4}'.format(roc_auc_score(y_test, probs)))
    
def show_basic_models(df):
    train_df = df.query('sample == 1').drop(['sample', 'client_id'], axis=1)

    X = train_df.drop(['default'], axis=1).values
    y = train_df['default'].values

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=RANDOM_SEED)
    
    # models with default settings
    lr = LogisticRegression(max_iter=1000) # фиксирую максимальное кол-во итераций
    tree = DecisionTreeClassifier()
    rforest = RandomForestClassifier()


    models = [lr,tree,rforest]

    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        probs = model.predict_proba(X_test)
        probs = probs[:,1]

        # zero_division=0 to fix zero division Warning
        print('Results for:', model)
        show_metrics(y_test, y_pred, probs)
        print('-'*10)
        print()
        
    
def compute_selected_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)
    probs = probs[:,1]
    show_metrics(y_test, y_pred, probs)
    return y_pred, probs

In [113]:
path = '/kaggle/input/credit-scoring/'

In [114]:
train = pd.read_csv(path +'train.csv')
test = pd.read_csv(path +'test.csv')
sample = pd.read_csv(path +'sample_submission.csv')

In [115]:
print(train.info())
print()
print('Train size: ', train.shape)
print()
train.head()

In [116]:
print(test.info())
print()
print('Test size: ', test.shape)
print()
test.head(5)

In [117]:
print(sample.info())
print()
print('Sample size: ', sample.shape)
print()
sample.head(5)

## Описания полей
* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия “хорошей” работы
* bki_request_cnt - количество запросов клиента в БКИ о своей кредитной истории
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

In [118]:
# посмотрим на целевую переменную
sns.countplot(x='default', data=train);

Сразу бросается в глаза, что выборка несбалансированная.

In [119]:
# для удобства предварительной обработки объединим датасеты 
train['sample'] = 1   # train
test['sample'] = 0    # test
test['default'] = -1    

data = pd.concat([train, test], ignore_index=True)

# num of unique values, first 10 unique values, null values count, type
data_agg = data.agg({'nunique', lambda s: s.unique()[:10]})\
    .append(pd.Series(data.isnull().sum(), name='null'))\
    .append(pd.Series(data.dtypes, name='dtype'))\
    .transpose()

data_agg

In [120]:
print(data.info())
print()
print(data.shape)

## EDA

Мы видим, что в наборе даных есть 19 признаков и количество клиентов (строк) равно 110 148.

Пропуски есть только в столбце education.

Дубликатов нет.

У признаки client_id все значения уникальные.

Признак app_date имеет только 120 вариантов значений. Основные данные за период февраль-апрель 2014 года

Признак education содержит 5 категорий и также имеет незаполненные значения:
1. SCH (52%) - School;
2. GRD (31%) - Graduated (Master degree);
3. UGR (13%) - UnderGraduated (Bachelor degree);
4. PGR (1.7%) - PostGraduated;
5. ACD (0.3%) - Academic Degree.

In [121]:
sns.countplot(x='sex', data=data);

Признак sex выражен в виде 2 категорий:

Female (56%);
Male (44%);

In [122]:
data['sex'].value_counts()/data.shape[0]

In [123]:
fig, axes = plt.subplots(figsize = (14, 4))
sns.countplot(x='age', data=data, ax=axes);


Признак age представлен значениями, которые смещены влево:

Minimum 21
median 37
Mean 39.2
Maximum 72
Interquartile range (IQR) 18




In [124]:
data['age'].describe()

In [125]:
sns.countplot(x='car', data=data);

Признак car -  бинарный признак,и большинство заемщиков не имеют автомобиля.


In [126]:
sns.countplot(x='car_type', data=data);

car_type  - это бинарный признак, который показывает отечественный или иностранный автомобиль у заемщика. У большинства заемщиков отечественный автомобиль. 

In [127]:
plt.hist(train['decline_app_cnt'], bins=50);

Количество отказанных прошлых заявок  - decline_app_cnt имеет распределение с значительным смещением влево. Большинство значений  - нулевые, то есть большинству клиентов не было отказано по прошлым заявкам.

In [128]:
sns.countplot(x='good_work', data=data);

Анализируя признак good_work можно увидеть, что большинство заемщиков не имеют хорошей работы.

score_bki 93% значений уникальны, распределение нормальное.

In [129]:
plt.hist(train['score_bki'], bins=50);

In [130]:
# sns.countplot(x='home_address', data=train);


train['home_address']
train['work_address']


fig, ax =plt.subplots(1,2)
sns.countplot(train['home_address'], ax=ax[0])
sns.countplot(train['work_address'], ax=ax[1])
fig.show()

Признаки home_address и work_address  - категориальные признаки с 3 вариантами значений.

In [131]:
plt.hist(train['income'], bins=30);

income большой разброс значений от 1000 до 1000000; можно попробовать либо превратить в категориальный признак, либо прологарифмировать

In [132]:

train['sna']
train['first_time']


fig, ax =plt.subplots(1,2)
sns.countplot(train['sna'], ax=ax[0])
sns.countplot(train['first_time'], ax=ax[1])
fig.show()

Признаки sna и first_time -  категориальные, и имеют по 4 вариации значений.

In [133]:
sns.countplot(x='foreign_passport', data=train);

foreign_passport бинарный признак, 67% заемщиков не имеют заграничный паспорт

Default  - наш целевой признак. Бинарный признак с подавляющим большинством тех, кто возвращает кредит без проблем. Выборка несбалансированная, при моделировании нужно будет попробовать undersampling.

In [134]:
# признаки сгруппируем в три категории по типу их обработки (категориальные, бинарные и числовые) для дальнейшего удобного анализа. 

num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt']

cat_cols = ['education','work_address','home_address','region_rating','sna','first_time']

bin_cols = ['sex','car','car_type','good_work','foreign_passport']

## Количественные признаки

In [135]:
# посмотрим на распределение количественных признаков
for i in num_cols:
    plt.figure()
    sns.displot(data[i].dropna(), kde = False, rug=False) 
    plt.title(f'Distribution of {i}')
    plt.show()

In [136]:
# посмотрим на выбросы и распределение целевой переменной между количественными признаками
for col in num_cols:
    get_boxplot(data, col)

Предварительные выводы:
* чем ниже возраст, тем более вероятен дефолт 
* чем выше score_bki, тем более вероятен дефолт
* более высокий доход говорит о меньшей вероятности дефолта

## Категориальные признаки

Клиенты с более низким уровнем образования имеют бОльшую вероятность дефолта, хотя именно они и хотят взять кредит чаще всего.

Чем ниже рейтинг региона, тем чаще возникает дефолт по кредитам. 

In [137]:
data.education.value_counts().plot(
    kind="bar",
    figsize=(8,6),
    color="r",
    title='Distribution of clients by Education',
    xlabel='education level',
    ylabel='number of clients'
)

print("Пропущенные значения:", data.education.isna().sum())
print()

In [138]:
# заполним пропуски наиболее часто встречающимся значением
data.education = data.education.fillna("SCH")

In [139]:
# оценим доход от уровня образования
plt.figure(figsize=(15, 8))
plt.title('Distribution of Income by Education level')
sns.boxplot(
    x="education", 
    y="income", 
    data=data, 
    showfliers=False
)

In [140]:
# оценим влияние региона проживания на уровень образования
plt.figure(figsize=(15, 8))
plt.title('Distribution of Education level by Region')
sns.boxplot(
    x="education", 
    y="region_rating", 
    data=data, 
    showfliers=False
)

Люди с более высоким уровнем образования живут в регионах с более высоким рейтингом. И наоборот.

In [141]:
# посмотрим на распределение дефолтных состояний по различным признакам

plt.figure(figsize=[20, 20])
i = 1

for k in cat_cols:
    plt.subplot(4, 3, i)
    sns.barplot(
        x=k, 
        y='proportion', 
        hue='default',  
        data=data[[k, 'default']].value_counts(normalize=True).rename('proportion').reset_index()
    )
    plt.title('Clients default distribution according to\n' + k, fontsize=15)
    i += 1
plt.tight_layout()
plt.show()

In [142]:
# закодируем категориальные признаки

mappc = {}
label_encoder = LabelEncoder()
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col])
    mappc[col] = dict(enumerate(label_encoder.classes_))
    
print(mappc)

## Бинарные признаки

Женщины чаще берут кредиты, чем мужчины. Относительное количество дефолтов при этом практически одинаковое.

Заемщики, у которых есть машина, более надежны. 

Заемщики, у которых иностранный автомобиль, более надежны, чем заемщики с отечественными машинами.

Заемщики с хорошей работой и заграничным паспортом возвращают долг чаще, чем без.

In [143]:
# посмотрим на распределение дефолтных состояний по различным признакам

plt.figure(figsize=[20, 20])
i = 1

for k in bin_cols:
    plt.subplot(4, 3, i)
    sns.barplot(
        x=k,
        y='proportion',
        hue='default',
        data=data[[k, 'default']].value_counts(normalize=True).rename('proportion').reset_index()
    )
    plt.title('Clients default distribution according to\n' + k, fontsize=15)
    i += 1
plt.tight_layout()
plt.show()

In [144]:
# закодируем бинарные признаки

mapp = {}
label_encoder = LabelEncoder()
for col in bin_cols:
    data[col] = label_encoder.fit_transform(data[col])
    mapp[col] = dict(enumerate(label_encoder.classes_))
    
print(mapp)

## Корреляционный анализ

Существенная корреляция между домашним адресом и местом работы. 

Сильная зависимость между car, car_type. 

Есть довольно сильная обратная зависимость между sna, first_time. 

In [173]:
plt.title('Correlation Matrix of dataset features')
plt.rcParams['figure.figsize'] = (40,40)
sns.heatmap(data.corr(), vmin=-1, vmax=1, annot = True)

In [175]:
plt.title('Correlation Matrix of dataset features (numerical)')
plt.rcParams['figure.figsize'] = (10,10)
sns.heatmap(data[num_cols].corr(), vmin=-1, vmax=1, annot = True)

In [147]:
plt.title('Correlation Matrix of dataset features (categorical)')
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data[cat_cols].corr(), vmin=-1, vmax=1, annot = True)

In [148]:
plt.title('Correlation Matrix of dataset features (binary)')
plt.rcParams['figure.figsize'] = (5,5)
sns.heatmap(data[bin_cols].corr(), vmin=-1, vmax=1, annot = True)

## Наивная модель


In [149]:
# временно удалим столбец,
# вернемся к его обработке при создании новых признаков
df = data.drop(['app_date'], axis=1)

In [150]:
show_basic_models(df)

## Создание признаков

### Декомпозиция адресов

Как мы установили ранее, home_address and work address обладают сильной корреляцией.

In [151]:
# получим кол-во дней от "начала эпохи" датасета
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
data_min = min(data['app_date'])
data['days'] = (data['app_date'] - data_min).dt.days.astype('int')
data['day'] = data['app_date'].dt.day
data['month'] = data['app_date'].dt.month

data.drop(['app_date'],  axis = 1, inplace = True)

# средний доход для конкретного возраста
mean_income = data.groupby('age')['income'].mean().to_dict()
data['mean_income_age'] = data['age'].map(mean_income)

# максимальный доход для конкретного возраста
max_income = data.groupby('age')['income'].max().to_dict()
data['max_income_age'] = data['age'].map(max_income)

# минимальный доход для конкретного возраста
min_income = data.groupby('age')['income'].min().to_dict()
data['min_income_age'] = data['age'].map(min_income)

# нормализуем доход
data["normalized_income"] = abs((data.income - data.mean_income_age)/data.max_income_age)
data.drop(['mean_income_age', 'max_income_age'],  axis = 1, inplace = True)

# среднее кол-во запросов в БКИ по конкретному возрасту
mean_bki = data.groupby('age')['bki_request_cnt'].mean().to_dict()
data['mean_requests_age'] = data['age'].map(mean_bki)

# максимальное кол-во запросов в БКИ по конкретному возрасту
max_bki = data.groupby('age')['bki_request_cnt'].max().to_dict()
data['max_requests_age'] = data['age'].map(max_bki)

# нормализуем requests
data["normalized_req"] = abs((data.bki_request_cnt - data.mean_requests_age)/data.max_requests_age)
data.drop(['mean_requests_age', 'max_requests_age'],  axis = 1, inplace = True)

# среднее кол-во запросов в БКИ в зависимости от дохода
mean_bki_inc = data.groupby('income')['bki_request_cnt'].mean().to_dict()
data['mean_requests_income'] = data['income'].map(mean_bki_inc)

# средний доход по региону
mean_income_rat = data.groupby('region_rating')['income'].mean().to_dict()
data['mean_income_region'] = data['region_rating'].map(mean_income_rat)

data.drop(['income'],  axis = 1, inplace = True)

# сократим размерность матрицы без потери информации
# 0 - нет машины, 1 - есть отечественна машина, 2 - есть иномарка
data['car_comb'] = data['car'] + data['car_type']
data['car_comb'] = data['car_comb'].astype('category')
data.drop(['car', 'car_type'], axis=1, inplace=True)

# возраст разделим на четыре категории
data['age_cat'] = data['age'].map(lambda x: age_to_cat(x))
data.drop('age', axis=1, inplace=True)

label_encoder = LabelEncoder()
data['age_cat'] = label_encoder.fit_transform(data['age_cat'])
    
# Sort out decline_app_cnt and bki_request_cnt by groups:
data['decline_cat'] = data['decline_app_cnt'].apply(lambda x: 4 if x >= 4 else x) 
data['bki_request_cat'] = data['bki_request_cnt'].apply(lambda x: 6 if x >= 6 else x)
data.drop(['decline_app_cnt', 'bki_request_cnt'], axis=1, inplace=True)

# Декомпозиция адресов

# вытащим два столбца из датасета
data_addresses = data[['work_address', 'home_address']].values

# создадим Scaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_addresses)

# У нас два вектора. Сократим до одного, оставив наиболее значимую информацию.
pca = PCA(n_components=1)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)
data['pca_address'] = pca_data
data['pca_address'] = data['pca_address'] + 5
data['pca_address'] = data['pca_address'].apply(lambda x: np.log(x) + 1)

# удалим ненужные столбцы
data.drop(['home_address','work_address'], axis=1, inplace=True)

data = data.fillna(data.mean())

In [152]:
# обновим списки признаков в переменных для обработки
# exclude day, month (high correlation)
num_cols = [
    'score_bki',
    'days',
    'min_income_age',
    'normalized_income',
    'normalized_req',
    'mean_requests_income',
    'mean_income_region',
]
cat_cols = [
    'education',
    'region_rating',
    'sna',
    'first_time',
    'car_comb',
    'age_cat',
    'decline_cat',
    'bki_request_cat',
    'pca_address',
]
bin_cols = [
    'sex',
    'good_work',
    'foreign_passport',
]

## Выбросы и стандартизация

In [153]:
for col in num_cols:
    median = data[col].median()
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)
    perc25 = data[col].quantile(0.25)
    perc75 = data[col].quantile(0.75)
    
    print("Column: ", col)
    print(' 25%: {:.4},\n'.format(perc25), '75%: {:.4},\n'.format(perc75),
          "IQR: {:.4}, \n".format(IQR), "Borderline: [{f:.4}, {l:.4}].\n".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
    print()
    
    # заменяем значения выбросов 
    data[col] = np.where(data[col] > (perc75 + 1.5*IQR), (perc75 + 1.5*IQR), data[col])
    data[col] = np.where(data[col] < (perc25 - 1.5*IQR), (perc25 - 1.5*IQR), data[col])

In [154]:
scaler = RobustScaler()
data[num_cols] = scaler.fit_transform(data[num_cols].values)

In [155]:
data.info()

In [156]:
data.head()

## Моделирование после добавления признаков


In [157]:
show_basic_models(data)

## Промежуточные выводы

Бинарные признаки:
* Car и car_type сильно взаимозависимы. Объединены в один признак car_comb с тремя характеристиками.
* Количество должников среди мужчин и женщин примерно одинаковое, но женщины берут кредиты чаще.

Категориальные признаки:
* Люди со средним образованием и ниже возвращают кредиты реже, чем люди с высшим образованием.
* Чем больше отношений у клиента с другими клиентами в банке - тем лучше и меньше просроченных кредитов.
* Люди с высшим образованием живут в регионах с более высоким рейтингом.
* Чем выше рейтинг региона, тем ниже риск дефолта.

Numerical:
* score_bki имеет распределение, близкое к нормальному
* В данных есть выбросы. Устранены через преобразование признаков в категориальные, логарифмирование или с использованием Scaler
* Между количественными признаками нет сильных корреляций
* Наличие иномарки коррелирует с уровнем дохода
* Количество связей с другими клиентами банка коррелирует с наличием заграничного паспорта

Наиболее статистически значимые признаки:
* sna
* pca_address (home & work addresses)
* first_time
* score_bki
* mean_income_region

Поскольку мы имеем достаточно много неочевидных корреляций между признаками, стоит попробовать использовать logistic regressions (при этом descicion tree models показали довольно слабые результаты).

## Моделирование

Сравним logisticRegression со стандартными настройками и настройкой class_weight='balanced'.

In [158]:
train_df = data.query('sample == 1').drop(['sample', 'client_id'], axis=1)
test_df = data.query('sample == 0').drop(['sample', 'client_id'], axis=1)

X = train_df.drop(['default'], axis=1).values
y = train_df['default'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED)

In [159]:
# default model

lr = LogisticRegression(max_iter=500)
y_pred, probs = compute_selected_model(lr)


In [160]:
# penalty=none is stronger than penalty=l2

lr_penalty_none = LogisticRegression(penalty='none', max_iter=1000)
y_pred, probs = compute_selected_model(lr_penalty_none)


In [161]:
# multi_class=multinominal is weaker than auto or ovr (them equal)

lr_ovr = LogisticRegression(penalty='l2', max_iter=1000, multi_class='ovr')
y_pred, probs = compute_selected_model(lr_ovr)


In [162]:
# saga is weaker than sag with equal max_iter
# both sag and saga weaker than default solver

lr_saga = LogisticRegression(penalty='l2', max_iter=1500, solver='saga')
y_pred, probs = compute_selected_model(lr_saga)


In [163]:
# balanced is weaker than default settings in roc_auc
# but f1_score is significant stronger

lr_balanced = LogisticRegression(class_weight='balanced', max_iter=500)
y_pred, probs = compute_selected_model(lr_balanced)


## Оценка ROC AUC и других метрик

In [164]:
# best LogReg model from previous chapter with balanced class weights

lr_penalty_none = LogisticRegression(penalty='none', max_iter=1000)
y_pred, probs = compute_selected_model(lr_penalty_none)

In [165]:
fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [166]:
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default', 'default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

## Подбор гиперпараметров

In [167]:
train_data = data.query('sample == 1').drop(['sample', 'client_id'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'client_id'], axis=1)

X_train = train_data.drop(['default'], axis=1)
y_train = train_data.default.values
X_test = test_data.drop(['default'], axis=1)
y_test = test_data.default.values

In [168]:
params = {'C' : np.logspace(-4, 4, 20)}

model = LogisticRegression(penalty='none', max_iter=1000, n_jobs=4)
model.fit(X_train, y_train)

clf = GridSearchCV(model, params, cv=5, verbose=3)

best_model = clf.fit(X_train, y_train)

print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [169]:
y_pred = best_model.predict_proba(X_test)
results_df = pd.DataFrame(data={'client_id':test['client_id'], 'default':y_pred[:,1]})
results_df.to_csv('submission.csv', index=False)
results_df

#### ROC AUC = 0.7393

## Выводы

По итогу мы использовали logisticRegression со сбалансированными признаками - такая модель демонстрирует более качественные результаты по всем метрикам.